# Bibliotecas

In [1]:
import spacy
import pandas as pd
import unicodedata

# Normalizador

In [2]:
def norm(s):
    if not isinstance(s, str):
        return s
    s = s.lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn')
    return s

# Carregar dados

In [3]:
df_class_words = pd.read_csv('sentilex_new.csv', sep = ';')

# Entrada do texto

In [4]:
text1 = """O evento começou pontualmente às 10h da manhã, conforme anunciado no cronograma. O palestrante principal falou por exatos 45 minutos, cobrindo os tópicos listados na descrição da palestra. Após a apresentação, os participantes foram direcionados ao salão adjacente para o intervalo. A organização informou que a próxima sessão começará em 15 minutos."""

In [5]:
text2 = """A experiência de hoje foi absolutamente fantástica! Desde o primeiro momento, a equipe demonstrou um profissionalismo e uma simpatia incríveis, superando todas as minhas expectativas. O produto não é apenas bom, é maravilhoso e funciona perfeitamente. A qualidade é evidente em cada detalhe. Estou genuinamente impressionado e muito satisfeito com o resultado. Com certeza recomendo a todos; foi uma escolha excelente e valeu cada centavo!"""

In [6]:
text3 = """Minha experiência hoje foi absolutamente terrível. Desde o início, o atendimento demonstrou um descaso e uma falta de profissionalismo inacreditáveis, frustrando todas as minhas expectativas. O produto não é apenas ruim, é péssimo e falha constantemente. A baixa qualidade é evidente em cada detalhe. Estou genuinamente decepcionado e muito insatisfeito com o resultado. Com certeza não recomendo a ninguém; foi uma escolha horrível e um desperdício de dinheiro."""

In [7]:
ig_text = "É bem comum as pessoas terem gostos próprios."

# Tokenização

In [8]:
nlp = spacy.load('pt_core_news_sm')
text = ig_text
doc = nlp(text)

# Extraindo os tokens
tokens = [token.text for token in doc]
df_text_tokens = pd.DataFrame(tokens)
df_text_tokens.columns = ['word']

# Normalização

In [9]:
df_class_words["word"] = df_class_words["word"].apply(norm)
df_text_tokens["word"] = df_text_tokens["word"].apply(norm)

# Mesclando dados

In [10]:
merged = df_text_tokens.merge(
    df_class_words[['word', 'polarity_n0','polarity_n1']],
    on='word',
    how='left'
)

# Removendo valores nulos

In [11]:
merged = merged.dropna(subset=['polarity_n0'])

# Contando polaridades

In [12]:
contagem = merged['polarity_n0'].value_counts().reindex([-1, 0, 1], fill_value=0)
contagem

polarity_n0
-1    0
 0    0
 1    0
Name: count, dtype: int64

# Checando o resultado

In [13]:
if (contagem == contagem.max()).sum() > 1:
    resultado = None     # empate
else:
    resultado = contagem.idxmax()
print(resultado)
if resultado == 1:
    print("positivo")
elif resultado == -1:
    print("negativo")
else :
    print("neutro")

None
neutro
